In [1]:
import sys, os, time
os.environ["CUDA_VISIBLE_DEVICES"]="0"
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
from torch.utils.data import Dataset, DataLoader
import gym
import numpy as np
%matplotlib notebook
#%matplotlib tk
import matplotlib.pyplot as plt
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
import foundation as fd
from foundation import models
from foundation import util
from foundation import train
#from foundation.util import replicate, Cloner

np.set_printoptions(linewidth=120)

In [19]:
info = torch.load('rl/temp_xy.pth.tar')
info.keys()

dict_keys(['y', 'm_old', 'x', 'm_new', 'l_new', 'l_old'])

In [20]:
X, Y = info['x'], info['y']
X.shape, Y.shape

(torch.Size([2050, 4]), torch.Size([2050, 1]))

In [21]:
mn, mo = info['m_new'], info['m_old']
print(list(mn.parameters()))
print(list(mo.parameters()))

[Parameter containing:
tensor([[ -6.0671, -15.6417,   1.6947,  -0.5189]]), Parameter containing:
tensor([5.8610])]
[Parameter containing:
tensor([[-0.4195,  0.3034,  0.2341,  0.3355]]), Parameter containing:
tensor([0.1772])]


In [22]:
info['l_old'], info['l_new']

(tensor(28.8512), tensor(5.7119))

In [3]:
N,M,D = 20, 5, 3
with torch.no_grad():
    A = torch.randn(N,M)
    f = nn.Linear(M,D)
    b = f(A)
    m = nn.Linear(M,D)

In [5]:
F.mse_loss(m(A), b)

tensor(1.1172, grad_fn=<MseLossBackward>)

In [6]:
util.solve(A,b,out=m)

Linear(in_features=5, out_features=3, bias=True)

In [7]:
F.mse_loss(m(A), b)

tensor(3.1220e-14, grad_fn=<MseLossBackward>)

In [25]:
shapes, settings = nets.plan_deconv(out_shape=(3,160,120), channels=[8, 16, 32, 64], kernels=4, strides=2)
shapes

[(8, 10, 7), (16, 20, 15), (32, 40, 30), (64, 80, 60), (3, 160, 120)]

In [26]:
ms = nn.ModuleList([nn.ConvTranspose2d(**s) for s in settings])
ms

ModuleList(
  (0): ConvTranspose2d(8, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), output_padding=(0, 1))
  (1): ConvTranspose2d(16, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (2): ConvTranspose2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (3): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
)

In [29]:
X = torch.randn(1, *shapes[0])

In [30]:
x = X
print(x.size())
for m in ms:
    x = m(x)
    print(x.size())

torch.Size([12, 8, 10, 7])
torch.Size([12, 16, 20, 15])
torch.Size([12, 32, 40, 30])
torch.Size([12, 64, 80, 60])
torch.Size([12, 3, 160, 120])


In [22]:
print(x.shape)
print(m)

torch.Size([12, 8, 10, 8])
ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))


In [45]:
shapes, settings = nets.plan_conv((3,160,120), channels=[8, 16, 32, 64], kernels=3, strides=1, factors=2)
shapes

[(3, 160, 120), (8, 80, 60), (16, 40, 30), (32, 20, 15), (64, 10, 8)]

In [46]:
ms = nn.ModuleList([nn.Conv2d(**s) for s in settings])
ms

ModuleList(
  (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [47]:
X = torch.randn(1, *shapes[0])

In [48]:
x = X
print(x.size())
for m in ms:
    x = m(x)
    print(x.size())

torch.Size([1, 3, 160, 120])
torch.Size([1, 8, 160, 120])
torch.Size([1, 16, 160, 120])
torch.Size([1, 32, 160, 120])
torch.Size([1, 64, 160, 120])


In [ ]:
ms = nn.ModuleList([nn.Conv2d(**s) for s in settings])

In [3]:
class OU_Process:
    def __init__(self, K=3, std=0.5, theta=0.5):
        self.K = K
        self.std = std
        self.theta = theta
        
        self.x = torch.randn(K) * self.std
        
    def __call__(self, t, state):
        return self.theta * self.x + torch.randn(self.K) * self.std
    
def fig2data ( fig ):
    """
    @brief Convert a Matplotlib figure to a 4D numpy array with RGBA channels and return it
    @param fig a matplotlib figure
    @return a numpy 3D array of RGBA values
    """
    # draw the renderer
    fig.canvas.draw ( )
 
    # Get the RGBA buffer from the figure
    w,h = fig.canvas.get_width_height()
    buf = np.fromstring ( fig.canvas.tostring_argb(), dtype=np.uint8 )
    buf.shape = ( w, h,4 )
 
    # canvas.tostring_argb give pixmap in ARGB mode. Roll the ALPHA channel to have it in RGBA mode
    buf = np.roll ( buf, 3, axis = 2 )
    return buf

In [4]:
#env.alpha = 5.
#env.beta = 5.
#env.gamma = 0.1

In [5]:
env.reset()#(-torch.ones(2)*np.pi/2, torch.zeros(2))
#policy = OU_Process(K=env.N, std=0.5, theta=0.7)
policy = lambda t, x: torch.rand(env.spec.action_space.size)*2 - 1
plt.ioff()

In [6]:
rgb, lbl = env.render(label=True)
rgb.shape, lbl.shape

((200, 200, 3), (200, 200))

In [7]:
env.obs

tensor([-0.9174,  0.9359, -0.3980,  0.3524, -0.0811,  2.2122, -0.5983,
         0.3812,  0.0092, -0.0228])

In [8]:
plt.ion()
plt.figure()
plt.imshow(rgb, interpolation='none')

plt.figure()
plt.imshow(lbl, interpolation='none')

In [9]:
plt.ion()
figax = plt.subplots(figsize=(3,3))
fig, ax = figax
ax.set_xticks([])
ax.set_yticks([])

[]

In [10]:
for t in range(1000):
    
    action = policy(t, None) #+ 1
    env.step(action)
    
    ax.set_title('{:.3f} {:.3f}'.format(action[0].item(), action[1].item()))
    
    env.render('onscreen', figax=figax)
    plt.pause(0.002)
    ax.cla()
    #break

In [10]:
plt.show()

In [8]:
rgb = fig2data(fig)
rgb.shape

/home/fleeb/anaconda3/envs/cse/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


(409, 400, 4)

In [7]:
plt.figure()
#plt.axis('off')
plt.imshow(env.render('rgb'))
plt.show()

In [7]:
env.get_ctrl_freq()

60.0